# Development notebook
# Allow for image level parameter input: added to *digipath_toolkit.py*

## *get_patch_location_array_for_image_level(run_parameters)*

## *get_patch_locations_preview_imagefor_image_level(run_parameters)*

In [ ]:
import time
import os
import sys

from collections import OrderedDict
import argparse

import numpy as np
import pandas as pd
import yaml

from skimage.filters import threshold_otsu
from skimage.color import rgb2lab

from PIL import ImageDraw
from PIL import TiffImagePlugin as tip

import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import *

data_dir = '../../DigiPath_MLTK_data/Aperio'
file_type_list=['.svs', '.tif', '.tiff']
fs_od = get_file_size_ordered_dict(data_dir, file_type_list)
list_number = 0
for k, v in fs_od.items():
    print('%3i %30s: %i'%(list_number, k,v))
    list_number += 1

In [ ]:
#                         test data file for threshold_otsu
data_dir = '../../DigiPath_MLTK_data/Aperio'
image_file_name = 'CMU-1.svs'

run_parameters = dict()

run_parameters['wsi_filename'] = os.path.join(data_dir, image_file_name)
print('Image File:\n', run_parameters['wsi_filename'])

run_parameters['thumbnail_divisor'] = 10
run_parameters['patch_select_method'] = 'threshold_otsu' # 'threshold_rgb2lab'
run_parameters['patch_height'] = 224
run_parameters['patch_width'] = 224
run_parameters['threshold'] = 0

"""
                output & time test:                CALL - - get_patch_location_array()
"""
t0 = time.time()
patch_location_array = get_patch_location_array(run_parameters)
tt = time.time() - t0

print('%i images found\t%0.3f s\n'%(len(patch_location_array), tt))

print('type(patch_location_array) =',type(patch_location_array), '\t', len(patch_location_array),'\n')
for d in patch_location_array:
    print(d)

In [ ]:
def get_patch_location_array_for_image_level_NB(run_parameters):
    """ Usage: patch_location_array = get_patch_location_array_for_image_level(run_parameters)
        using 'patch_select_method", find all upper left corner locations of patches
        that won't exceed image size givin the 'patch_height' and 'patch_width'

    Args (run_parameters):  python dict.keys()
                                wsi_filename:           file name (with valid path)
                                patch_height:           patch size = (patch_width, patch_height)
                                patch_width:            patch size = (patch_width, patch_height)
                                thumbnail_divisor:      wsi_image full size divisor to create thumbnail image
                                patch_select_method:    'threshold_rgb2lab' or 'threshold_otsu'
                                threshold:              sum of thresholded image minimimum (default = 0)
                                image_level:            openslide image pyramid level 0,1,2,...
    Returns:
        patch_location_array

    """
    patch_location_array = []

    wsi_filename = run_parameters['wsi_filename']
    thumbnail_divisor = run_parameters['thumbnail_divisor']
    patch_select_method = run_parameters['patch_select_method']
    patch_height = run_parameters['patch_height']
    patch_width = run_parameters['patch_width']
    
    if 'threshold' in run_parameters:
        threshold = run_parameters['threshold']
    else:
        threshold = 0
        
    if 'image_level' in run_parameters:
        image_level = run_parameters['image_level']
    else:
        image_level = 0

    #                     OpenSlide open                      #
    os_im_obj = openslide.OpenSlide(wsi_filename)
    
    level_count = os_im_obj.level_count
    level_downsamples = os_im_obj.level_downsamples
    obj_level_diminsions = os_im_obj.level_dimensions
    
    print('%i levels'%(level_count))
    for k in range(level_count):
        print(k, level_downsamples[k], obj_level_diminsions[k])
    
    pixels_height = obj_level_diminsions[image_level][1]
    rows_fence_array = get_fence_array(patch_length=patch_height, overall_length=pixels_height)

    pixels_width = obj_level_diminsions[image_level][0]
    cols_fence_array = get_fence_array(patch_length=patch_width, overall_length=pixels_width)

    thumbnail_size = (pixels_width // thumbnail_divisor, pixels_height // thumbnail_divisor)
    small_im = os_im_obj.get_thumbnail(thumbnail_size)
    os_im_obj.close()
    #                     OpenSlide close                     #
    print('sizes\trows_fence_array: %i\tcols_fence_array: %i'%(len(rows_fence_array),len(cols_fence_array)))
    print('small_im\th: %i\tw: %i\t'%(pixels_height, pixels_width), thumbnail_size, '\t', small_im.size)
    mask_im = get_sample_selection_mask(small_im, patch_select_method)
    print('type(mask_im)',type(mask_im))
    it_rows = zip(rows_fence_array[:, 0] // thumbnail_divisor,
                  rows_fence_array[:, 1] // thumbnail_divisor,
                  rows_fence_array[:, 0])

    lft_cols = cols_fence_array[:, 0] // thumbnail_divisor
    rgt_cols = cols_fence_array[:, 1] // thumbnail_divisor
    cols_array = cols_fence_array[:, 0]
    do_print = True
    for tmb_row_top, tmb_row_bot, row_n in it_rows:
        it_cols = zip(lft_cols, rgt_cols, cols_array)
        for tmb_col_lft, tmb_col_rgt, col_n in it_cols:
            if (mask_im[tmb_row_top:tmb_row_bot, tmb_col_lft:tmb_col_rgt]).sum() > threshold:
                patch_location_array.append((row_n, col_n))
                if do_print == True:
                    do_print = False
                    print('mask(%i:%i, %i:%i)'%(tmb_row_top,tmb_row_bot,tmb_col_lft,tmb_col_rgt))
    print('mask(%i:%i, %i:%i)'%(tmb_row_top,tmb_row_bot,tmb_col_lft,tmb_col_rgt))
    
    return patch_location_array

In [ ]:
for im_lvl in range(3):
    run_parameters['image_level'] = im_lvl
    patch_location_array = get_patch_location_array_for_image_level(run_parameters)
    print(len(patch_location_array), '\n')

In [ ]:
def get_patch_locations_preview_imagefor_image_level_NB(run_parameters):
    """ Usage: 
    mask_image, thumb_preview, patch_location_array = get_patch_locations_preview_imagefor_image_level(run_parameters)
    get the images and data needed to display where the patches are for the input parameters

    Args (run_parameters):  python dict.keys()
                                wsi_filename:           file name (with valid path)
                                border_color:           patch-box representation color 'red', 'blue' etc
                                patch_height:           patch size = (patch_width, patch_height)
                                patch_width:            patch size = (patch_width, patch_height)
                                thumbnail_divisor:      wsi_image full size divisor to create thumbnail image
                                patch_select_method:    'threshold_rgb2lab' or 'threshold_otsu'
                                threshold:              sum of thresholded image minimimum (default = 0)
                                image_level:            openslide image pyramid level 0,1,2,...
    Returns:
        mask_image:             black & white image of the mask
        thumb_preview:          thumbnail image with patch locations marked
        patch_location_array:   list of patch locations used [(row, col), (row, col),... ]

    """
    wsi_filename = run_parameters['wsi_filename']
    patch_select_method = run_parameters['patch_select_method']
    thumbnail_divisor = run_parameters['thumbnail_divisor']
    patch_height = run_parameters['patch_height'] // thumbnail_divisor - 1
    patch_width = run_parameters['patch_width'] // thumbnail_divisor - 1
    border_color = run_parameters['border_color']
    
    if 'threshold' in run_parameters:
        threshold = run_parameters['threshold']
    else:
        threshold = 0
        
    if 'image_level' in run_parameters:
        image_level = run_parameters['image_level']
    else:
        image_level = 0
    
    #                     OpenSlide open                      #
    os_im_obj = openslide.OpenSlide(wsi_filename)
    
    print('\nimage_level',image_level)
    image_level = min((os_im_obj.level_count -1), image_level)
    print('image_level',image_level, '\n')
    
    level_count = os_im_obj.level_count
    level_downsamples = os_im_obj.level_downsamples
    obj_level_diminsions = os_im_obj.level_dimensions
    
    pixels_width = obj_level_diminsions[image_level][0]
    pixels_height = obj_level_diminsions[image_level][1]
        
    thumbnail_size = (pixels_width // thumbnail_divisor, pixels_height // thumbnail_divisor)
    thumb_preview = os_im_obj.get_thumbnail(thumbnail_size)
    os_im_obj.close()

    mask_image = get_sample_selection_mask(thumb_preview, patch_select_method)
    mask_image = tip.Image.fromarray(np.uint8(mask_image * 255), 'L')

    thumb_draw = ImageDraw.Draw(thumb_preview)
    patch_location_array = get_patch_location_array_for_image_level_NB(run_parameters)

    for r, c in patch_location_array[:]:
        ulc = (c // thumbnail_divisor, r // thumbnail_divisor)
        lrc = (ulc[0] + patch_width, ulc[1] + patch_height)
        thumb_draw.rectangle((ulc, lrc), outline=border_color, fill=None)

    return mask_image, thumb_preview, patch_location_array

In [ ]:
run_parameters['image_level'] = 0
run_parameters['thumbnail_divisor'] = 20

run_parameters['threshold'] = 0
run_parameters['border_color'] = 'blue'
t2 = time.time()
mask_im, prev_im, patch_array = get_patch_locations_preview_imagefor_image_level_NB(run_parameters)
tt2 = time.time() - t2
# sm_prv_im = prev_im.resize()
print('tt2 = %0.3f'%(tt2), 'prev_im.size:', prev_im.size)
display(prev_im)

In [ ]:
run_parameters['image_level'] = 1
run_parameters['thumbnail_divisor'] = run_parameters['thumbnail_divisor'] // (run_parameters['image_level'] + 1)

run_parameters['threshold'] = 0
run_parameters['border_color'] = 'blue'
t2 = time.time()
mask_im, prev_im, patch_array = get_patch_locations_preview_imagefor_image_level_NB(run_parameters)
tt2 = time.time() - t2
print('tt2 = %0.3f'%(tt2), 'prev_im.size:', prev_im.size)
display(prev_im)

In [ ]:
run_parameters['image_level'] = 2
run_parameters['thumbnail_divisor'] = run_parameters['thumbnail_divisor'] // (run_parameters['image_level'] + 1)

run_parameters['threshold'] = 0
run_parameters['border_color'] = 'blue'
t2 = time.time()
mask_im, prev_im, patch_array = get_patch_locations_preview_imagefor_image_level_NB(run_parameters)
tt2 = time.time() - t2
print('tt2 = %0.3f'%(tt2), 'prev_im.size:', prev_im.size)
display(prev_im)

In [ ]:
run_parameters['image_level'] = 2
run_parameters['thumbnail_divisor'] = run_parameters['thumbnail_divisor'] // (run_parameters['image_level'] + 1)

run_parameters['threshold'] = 0
run_parameters['border_color'] = 'blue'
t2 = time.time()
mask_im, prev_im, patch_array = get_patch_locations_preview_imagefor_image_level_NB(run_parameters)
tt2 = time.time() - t2
print('tt2 = %0.3f'%(tt2), 'prev_im.size:', prev_im.size)
display(prev_im)